# Crawling  vs  Parsing
둘은 하는 역할이 다르다.

- **Crawling**
    - web 상에서 필요한 데이터를 찾아다니는 bot.
    - Scrapy(web-spyder package)
        - 이 수업에서는 사용하지 않을 예정
        - 초보자에게는 유용한 package이다. 
    
- **Parsing**
    - 단어 단위로 쪼개서 필요한 내용을 추출하는 작업.
    - BeautifulSoup
        - Text를 받아서 DOM 형태로 만들어주는 역할

---

# Crawling 실습
- 지난주 배운 내용으로 할 수 있음.
- 예제 사이트: http://example.webscraping.com/places/default/index

In [2]:
from bs4 import BeautifulSoup
import requests
import time
import random

In [14]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36"}

In [1]:
# 만들어뒀던 download 함수

def download(method="get", url="", headers={}, params={}, retries=3):
    try:
        resp = requests.request(method, 
                                url, 
                                headers=headers,
                                params=params if method=="get" else {},
                                data = params if method=="post" else {}
                               )
        resp.raise_for_status()
        
    except requests.exceptions.HTTPError as e:
        if 500 <= resp.status_code < 600 and retries > 0:
            print("Retries:",retries)
            time.sleep(random.randint(1,5))
            return download(method, url, headers, params, retries-1)
            
        else:
            print(resp.status_code)
            print(resp.reason)
            print(resp.request.headers)
            print(resp.url)

    return resp

In [102]:
URLs = list()
visitedURLs = list()
URLs.append("http://example.webscraping.com/places/default/index")

while URLs:
    url = URLs.pop(0)
    
    resp = download("get", url, headers=headers)
    dom = BeautifulSoup(resp.text, "lxml")
    
    print(len(dom.find_all("a")))
    links = [_ for _ in dom.select("a")
            if _.has_attr("href")]
    print([_["href"] for _ in links])
    
    visitedURLs.append(url)

16
['#', '/places/default/user/register?_next=/places/default/index', '/places/default/user/login?_next=/places/default/index', '/places/default/index', '/places/default/search', '/places/default/view/Afghanistan-1', '/places/default/view/Aland-Islands-2', '/places/default/view/Albania-3', '/places/default/view/Algeria-4', '/places/default/view/American-Samoa-5', '/places/default/view/Andorra-6', '/places/default/view/Angola-7', '/places/default/view/Anguilla-8', '/places/default/view/Antarctica-9', '/places/default/view/Antigua-and-Barbuda-10', '/places/default/index/1']


In [5]:
# href 값이 정상적이지 않은 url이 섞여있다. 이걸 걸러내보겠습니다.
# request를 반복적으로 날리지 않기 위해서 temp 객체를 만들었다.

temp = [_["href"] for _ in links]
requests.utils.urlparse(temp[0]), \ # 죽은 url이다. (#은 현재페이지를 의미하므로 페이지 이동하지 않는 url임.) 
                                    # urlparse했을때 path가 없는 특징이 있다.
requests.utils.urlparse(temp[1])    # 살아있는 url이다. path가 있다. urlparse했을때 path가 있는 특징이 있다.

(ParseResult(scheme='', netloc='', path='', params='', query='', fragment=''),
 ParseResult(scheme='', netloc='', path='/places/default/user/register', params='', query='_next=/places/default/index', fragment=''))

In [8]:
requests.utils.urlparse(temp[0])[2], \ # 비어있다.
requests.utils.urlparse(temp[1])[2]    # 값이 있다.

('', '/places/default/user/register')

In [9]:
# 살아있는 url만 urljoin 시킨다.

temp = [requests.compat.urljoin(url, _["href"]) for _ in links
        if requests.utils.urlparse(_["href"])[2]]

temp

['http://example.webscraping.com/places/default/user/register?_next=/places/default/index',
 'http://example.webscraping.com/places/default/user/login?_next=/places/default/index',
 'http://example.webscraping.com/places/default/index',
 'http://example.webscraping.com/places/default/search',
 'http://example.webscraping.com/places/default/view/Afghanistan-1',
 'http://example.webscraping.com/places/default/view/Aland-Islands-2',
 'http://example.webscraping.com/places/default/view/Albania-3',
 'http://example.webscraping.com/places/default/view/Algeria-4',
 'http://example.webscraping.com/places/default/view/American-Samoa-5',
 'http://example.webscraping.com/places/default/view/Andorra-6',
 'http://example.webscraping.com/places/default/view/Angola-7',
 'http://example.webscraping.com/places/default/view/Anguilla-8',
 'http://example.webscraping.com/places/default/view/Antarctica-9',
 'http://example.webscraping.com/places/default/view/Antigua-and-Barbuda-10',
 'http://example.webscr

In [10]:
# 위의 코드를 살아있는 url만 수집하도록 수정함.

URLs = list()
visitedURLs = list()
URLs.append("http://example.webscraping.com/places/default/index")

while URLs:
    url = URLs.pop(0)
    
    resp = download("get", url, headers=headers)
    dom = BeautifulSoup(resp.text, "lxml")
    
    # 여기가 바뀐 부분.
    links = [requests.compat.urljoin(url, _["href"])  # urljoin으로 full url을 만든다.
             for _ in dom.select("a")
             if _.has_attr("href") and 
             requests.utils.urlparse(_["href"])[2]]   # 살아있는 url일 경우만 가져온다. 
    
    print([_ for _ in links])
    
    visitedURLs.append(url)

['http://example.webscraping.com/places/default/user/register?_next=/places/default/index', 'http://example.webscraping.com/places/default/user/login?_next=/places/default/index', 'http://example.webscraping.com/places/default/index', 'http://example.webscraping.com/places/default/search', 'http://example.webscraping.com/places/default/view/Afghanistan-1', 'http://example.webscraping.com/places/default/view/Aland-Islands-2', 'http://example.webscraping.com/places/default/view/Albania-3', 'http://example.webscraping.com/places/default/view/Algeria-4', 'http://example.webscraping.com/places/default/view/American-Samoa-5', 'http://example.webscraping.com/places/default/view/Andorra-6', 'http://example.webscraping.com/places/default/view/Angola-7', 'http://example.webscraping.com/places/default/view/Anguilla-8', 'http://example.webscraping.com/places/default/view/Antarctica-9', 'http://example.webscraping.com/places/default/view/Antigua-and-Barbuda-10', 'http://example.webscraping.com/plac

In [11]:
# 위의 코드를 처음보는 url만 추가하도록 수정함.

URLs = list()
visitedURLs = list() # 추가됨
URLs.append("http://example.webscraping.com/places/default/index")

while URLs:
    url = URLs.pop(0)
    visitedURLs.append(url) # 맨 아래에서 위로 자리만 옮김.

    resp = download("get", url, headers=headers)
    dom = BeautifulSoup(resp.text, "lxml")
    
    links = [requests.compat.urljoin(url, _["href"])  
             for _ in dom.select("a")
             if _.has_attr("href") and 
             requests.utils.urlparse(_["href"])[2]]   
    
    # 여기가 바뀐 부분.
    for link in links:
        if link not in URLs and link not in visitedURLs:
            URLs.append(link)
    
    break # 연습삼아 한번만 실행시켜보기 위해서 break를 사용함.

In [12]:
# 어떤 url들을 가져왔는지 보자.

URLs

['http://example.webscraping.com/places/default/user/register?_next=/places/default/index',
 'http://example.webscraping.com/places/default/user/login?_next=/places/default/index',
 'http://example.webscraping.com/places/default/search',
 'http://example.webscraping.com/places/default/view/Afghanistan-1',
 'http://example.webscraping.com/places/default/view/Aland-Islands-2',
 'http://example.webscraping.com/places/default/view/Albania-3',
 'http://example.webscraping.com/places/default/view/Algeria-4',
 'http://example.webscraping.com/places/default/view/American-Samoa-5',
 'http://example.webscraping.com/places/default/view/Andorra-6',
 'http://example.webscraping.com/places/default/view/Angola-7',
 'http://example.webscraping.com/places/default/view/Anguilla-8',
 'http://example.webscraping.com/places/default/view/Antarctica-9',
 'http://example.webscraping.com/places/default/view/Antigua-and-Barbuda-10',
 'http://example.webscraping.com/places/default/index/1']

In [103]:
# 위의 코드에서 천천히 request를 보내도록 time sleep을 추가했다.
# 한번에 너무 많은 request를 보내면 서버가 down될 수 있으므로.

URLs = list()
visitedURLs = list()
URLs.append("http://example.webscraping.com/places/default/index")

while URLs:
    url = URLs.pop(0)
    visitedURLs.append(url) 

    resp = download("get", url, headers=headers)
    dom = BeautifulSoup(resp.text, "lxml")
    
    links = [requests.compat.urljoin(url, _["href"])  
             for _ in dom.select("a")
             if _.has_attr("href") and 
             requests.utils.urlparse(_["href"])[2]]   
    
    for link in links:
        if link not in URLs and link not in visitedURLs:
            URLs.append(link)
    
    # 바뀐 부분.
    print(len(URLs), len(visitedURLs))
    time.sleep(random.randint(5, 10)) 

14 1
15 2
14 3
15 4
24 5
28 6
36 7
46 8
49 9
53 10
59 11
62 12
65 13
67 14
80 15
79 16
78 17
77 18
76 19


KeyboardInterrupt: 

In [21]:
# 위의 코드에서 속도를 개선했다.
# 1) response error가 났을 때는, dom 이하의 코드를 실행하지 않고 넘어가게 했다.
# 2) response error가 났을 때는, URLs의 맨 마지막에 추가해서 나중에 다시 시도하도록 만들었다.
# 3) request를 보낼 사이트가 직전에 request를 보냈던 사이트와 같은 곳일 경우에만 time sleep을 걸어줬다.

URLs = list()
visitedURLs = list()
URLs.append("http://example.webscraping.com/places/default/index")

lastURL = ""

while URLs:
    url = URLs.pop(0)
    visitedURLs.append(url) 

    resp = download("get", url, headers=headers)
    
    # 1), 2) 추가된 부분.
    if not resp:
        URLs.append(url)
        continue
    
    dom = BeautifulSoup(resp.text, "lxml")
    
    links = [requests.compat.urljoin(url, _["href"])  
             for _ in dom.select("a")
             if _.has_attr("href") and 
             requests.utils.urlparse(_["href"])[2]]   
    
    for link in links:
        if link not in URLs and link not in visitedURLs:
            URLs.append(link)
    
    print(len(URLs), len(visitedURLs))
    
    # 3) 추가된 부분.
    if requests.utils.urlparse(url)[1]  == lastURL:
        time.sleep(random.randint(5, 10))
    
    lastURL = requests.utils.urlparse(url)[1] 

14 1
15 2
14 3
15 4
24 5
28 6
36 7
46 8
49 9
53 10
59 11
62 12
65 13
67 14
80 15
79 16
78 17
77 18
76 19
75 20
74 21
122 22
126 23
138 24
144 25
146 26
148 27
150 28
149 29
148 30
147 31
198 32
197 33
196 34
195 35
194 36
198 37
200 38
206 39
208 40
210 41
212 42
211 43
210 44
209 45
266 46
273 47
275 48
279 49
282 50
284 51
286 52
290 53
289 54
288 55
287 56
312 57
311 58
310 59
309 60
314 61
322 62
321 63
320 64
319 65
328 66
332 67
337 68
341 69
340 70
339 71
338 72
376 73
375 74
374 75
373 76
377 77
376 78
375 79
374 80
373 81
372 82
371 83
379 84
382 85
384 86
386 87
392 88
394 89
396 90
398 91
400 92
402 93
403 94
406 95
405 96
404 97
406 98
408 99
411 100
414 101
413 102
415 103
417 104
419 105
421 106
424 107
428 108
427 109
433 110
437 111
439 112
442 113
444 114
446 115
448 116
451 117
453 118
455 119
459 120
461 121
463 122
465 123
467 124
470 125
474 126
473 127
475 128
477 129
479 130
483 131
485 132
487 133
489 134
491 135
493 136
492 137
494 138
496 139
495 140
497 141
4

KeyboardInterrupt: 

---

# Focused Crawling
특정 조건에 맞춰서 crawling하는 방법 

## 1. Focused Crawling - domain
- 특정 도메인 내에서만 crawling할때 사용하는 방법
- 예) 네이버에서 네이버 블로그만 crawling할 때

In [22]:
# 위의 코드에서 동일한 사이트로의 링크만 가져오는 조건을 추가하겠습니다. (Focused Crawling - domain)
# 다른 예제 사이트에서 하겠습니다.

URLs = list()
visitedURLs = list()
# URLs.append("http://example.webscraping.com/places/default/index")
URLs.append("http://pythonscraping.com/")

lastURL = ""

baseURL = requests.utils.urlparse(URLs[0])[1] 

while URLs:
    url = URLs.pop(0)
    visitedURLs.append(url) 

    resp = download("get", url, headers=headers)
    
    if not resp:
#         URLs.append(url)
        continue
    
    dom = BeautifulSoup(resp.text, "lxml")
    
    links = [requests.compat.urljoin(url, _["href"])  
             for _ in dom.select("a")
             if _.has_attr("href") and 
             requests.utils.urlparse(_["href"])[2]]
    
    for link in links:
        # 바뀐 부분
        if baseURL == requests.utils.urlparse(link)[1]:     
            if link not in URLs and link not in visitedURLs:
                URLs.append(link)
    
    print(len(URLs), len(visitedURLs))
    
    if requests.utils.urlparse(url)[1]  == lastURL:
        time.sleep(random.randint(5, 10))
    
    lastURL = requests.utils.urlparse(url)[1] 

3 1
2 2
1 3
6 4
6 5
5 6
6 7
5 8
4 9
8 10
7 11
6 12
404
Not Found
{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}
http://pythonscraping.com/wikiSpider.zip
4 14
3 15
2 16
1 17
0 18


## 2. Focused Crawling - depth
- crawler가 url을 너무 많이 타고 들어가는 것을 방지할 때 사용하는 방법
- 예) 구글 검색 결과에서 최대 3번까지만 url을 타고 들어가도록 제한하고 싶을 때

In [25]:
# 위의 코드에서 link의 link를 무한정 타고 가지 않도록 제한을 두겠습니다. (Focused Crawling - depth)
# useDepth 미만의 depth까지만 link를 타고 가겠습니다.

URLs = list()
visitedURLs = list()
URLs.append(("http://pythonscraping.com/", 1)) # URLs의 값을 (url, 숫자)로 된 튜플로 바꿨다.

lastURL = ""

baseURL = requests.utils.urlparse(URLs[0][0])[1] 
useBase = False # Focused Crawling - domain을 적용할지말지 옵션으로 만들었다.

# 추가된 부분
useDepth = 3

while URLs:
    urls = URLs.pop(0)
    
    # 바뀐 부분
    url = urls[0]
    depth = urls[1] + 1 
    
    visitedURLs.append(urls)

    resp = download("get", url, headers=headers)
    
    if not resp:
#         URLs.append(url)
        continue
    
    dom = BeautifulSoup(resp.text, "lxml")
    
    links = [requests.compat.urljoin(url, _["href"])  
             for _ in dom.select("a")
             if _.has_attr("href") and 
             requests.utils.urlparse(_["href"])[2]]
    
    for link in links:
        if useBase and baseURL == requests.utils.urlparse(link)[1]:     
            if link not in [_[0] for _ in URLs] and \
                link not in [_[0] for _ in visitedURLs]:
                URLs.append((link, depth))
                
        # 바뀐 부분
        if depth < useDepth:
            if link not in [_[0] for _ in URLs] and \
                link not in [_[0] for _ in visitedURLs]:
                URLs.append((link, depth))
    
    print(len(URLs), len(visitedURLs))
    
    if requests.utils.urlparse(url)[1]  == lastURL:
        time.sleep(random.randint(5, 10))
    
    lastURL = requests.utils.urlparse(url)[1] 

4 1
3 2
2 3
1 4
0 5


In [26]:
visitedURLs

[('http://pythonscraping.com/', 1),
 ('http://pythonscraping.com/node/5', 2),
 ('http://pythonscraping.com/node/15', 2),
 ('http://pythonscraping.com/blog', 2),
 ('http://shop.oreilly.com/product/0636920034391.do', 2)]

---

# 구글에서 실습

In [29]:
resp = download("get", "https://www.google.com/search?q=성소", headers=headers)
dom = BeautifulSoup(resp.text, "lxml")

In [37]:
searchResults = [(_.find_parent()["href"], 1) for _ in dom.select(".LC20lb")]

In [47]:

URLs = list()
visitedURLs = list()
URLs.extend(searchResults)

lastURL = ""

baseURL = requests.utils.urlparse(URLs[0][0])[1] 
useBase = False

useDepth = 3

while URLs:
    urls = URLs.pop(0)
    
    url = urls[0]
    depth = urls[1] + 1 
    print(url)
    
    visitedURLs.append(urls)

    resp = download("get", url, headers=headers)
    
    if not resp:
#         URLs.append(url)
        continue
    
    dom = BeautifulSoup(resp.text, "lxml")
    
    links = [requests.compat.urljoin(url, _["href"])  
             for _ in dom.select("a")
             if _.has_attr("href") and 
             requests.utils.urlparse(_["href"])[2]]
    
    for link in links:
        if useBase and baseURL == requests.utils.urlparse(link)[1]:     
            if link not in [_[0] for _ in URLs] and \
                link not in [_[0] for _ in visitedURLs]:
                URLs.append((link, depth))
        if depth < useDepth:
            if link not in [_[0] for _ in URLs] and \
                link not in [_[0] for _ in visitedURLs]:
                URLs.append((link, depth))
    
    print(len(URLs), len(visitedURLs))
    
    if requests.utils.urlparse(url)[1]  == lastURL:
        time.sleep(random.randint(5, 10))
    
    lastURL = requests.utils.urlparse(url)[1] 

https://namu.wiki/w/%EC%84%B1%EC%86%8C(%EC%9A%B0%EC%A3%BC%EC%86%8C%EB%85%80)
551 1
https://namu.wiki/w/%EC%84%B1%EC%86%8C
571 2
https://ko.wikipedia.org/wiki/%EC%84%B1%EC%86%8C_(%EA%B0%80%EC%88%98)
1038 3
https://1boon.daum.net/benter/sungso
1058 4
http://news.chosun.com/site/data/html_dir/2018/09/03/2018090301347.html
1068 5
https://twitter.com/hashtag/%EC%84%B1%EC%86%8C
1163 6
https://1boon.kakao.com/newsade/WJSN-SS
1172 7
https://news.sbs.co.kr/news/endPage.do?news_id=N1004917577
1349 8
https://www.mbcsportsplus.com/news/?mode=view&cate=&b_idx=99863462.000
1472 9
https://namu.wiki/w/%EB%82%98%EB%AC%B4%EC%9C%84%ED%82%A4:%EB%8C%80%EB%AC%B8
1471 10
https://namu.wiki/RecentChanges
1470 11


KeyboardInterrupt: 

---

### Queue를 가져오는 전략 
필요에 따라 전략은 선택하면 된다.

- Queue[0] -> BFS
    - queue stack의 앞에서부터 가져오는 방법
    - url.pop(0) 
    - 먼저 찾아둔 url을 먼저 scrap한다.
    
    
- Queue[-1] -> DFS
    - queue stack의 앞에서부터 가져오는 방법
    - url.pop(-1)
    - 나중에 찾은 url을 먼저 scrap한다.

---

# 실습 - 네이버
- 네이버 검색결과에서
    - useBase = True
    - baseUrl = blog.naver.com
    - useDepth = 0

In [80]:
naverUrl = "https://search.naver.com/search.naver"
params = "sm=top_hty&fbm=1&ie=utf8&query=성소"
params = params.split("&")
params = {_.split("=")[0] : _.split("=")[1]  for _ in params}

In [81]:
params

{'sm': 'top_hty', 'fbm': '1', 'ie': 'utf8', 'query': '성소'}

In [82]:
resp = download("get", naverUrl, params=params, headers=headers)
dom = BeautifulSoup(resp.text, "lxml")

In [83]:
[_.text for _ in dom.select(".section .type01 dt > a")]

['우주소녀(WJSN) 성소, 인스타그램 속 여전한 미모…한국 활동은 ...',
 '창해일성소',
 '성소 聖召',
 '수몰 위기서 다시 태어나다 [박윤정의 그레이트 이집트]',
 "우주소녀 성소, 고양이와 함께한 일상샷 '성소의 나이는?'",
 "우주소녀 성소, 독보적인 각선미 자랑 '나이는?'",
 '"장난치다…" 안동 경안中서 \'포르말린\' 누출',
 '경북경찰청, 안동 성소병원에 `마음동행센터` 개소',
 '부산성소병원',
 '성소 - 나무위키',
 '안동성소병원',
 '안동성소병원편도제거수술후기',
 '190704 군산 근대역사 사전탐방 및 우주소녀 은서 성소...',
 '마음속 성소(聖所)와 다락방',
 '교황 “교회는 성소를 필요로 합니다”',
 '아침묵상 / “이방인이 여호와의 집 성소에 들어가므로.”...']

In [86]:
naverResults = [(_["href"], 1) for _ in dom.select(".section .type01 dt > a")]

In [99]:
# 네이버 블로그는 HTML 안에 HTML이 있는 구조이다.
# HTML은 물리적으로 HTML을 내부에 심을 수가 있다.
# 이는 dom 안에 dom이 있는 구조이므로, parsing하기 위해서 내부에 있는 HTML을 가져와서 새롭게 dom을 만들어야한다.
ii
URLs = list()
visitedURLs = list()
URLs.extend(naverResults)

lastURL = ""

baseURL = "blog.naver.com"
useBase = True

useDepth = 0

while URLs:
    urls = URLs.pop(-1)
    
    url = urls[0]
    depth = urls[1] + 1 
    print(url)
    
    visitedURLs.append(urls)

    resp = download("get", url, headers=headers)
    
    if not resp:
        if 500 =< resp.stauts_code < 600
            URLs.append(url)
        continue
    
    dom = BeautifulSoup(resp.text, "lxml")
    
    # Iframe 처리
    for _ in [requests.compat.urljoin(url, _["src"])
             for _ in dom.select("iframe") 
              if _.has_attr("src") and 
                _.has_attr("id") and
                _["id"] == "mainFrame"]:
        iframe = download("get", _, headers=headers)
        iframeDom = BeautifulSoup(iframe.text, "lxml")
    
        links = [requests.compat.urljoin(url, _["href"])  
                 for _ in iframeDom.select("a")
                 if _.has_attr("href") and 
                 requests.utils.urlparse(_["href"])[2]]

        for link in links:
            if useBase and baseURL == requests.utils.urlparse(link)[1]:     
                if link not in [_[0] for _ in URLs] and \
                    link not in [_[0] for _ in visitedURLs]:
                    URLs.append((link, depth))
            if depth < useDepth:
                if link not in [_[0] for _ in URLs] and \
                    link not in [_[0] for _ in visitedURLs]:
                    URLs.append((link, depth))
    
    # Link 처리
    links = [requests.compat.urljoin(url, _["href"])  
             for _ in dom.select("a")
             if _.has_attr("href") and 
             requests.utils.urlparse(_["href"])[2]]
        
    for link in links:
        if useBase and baseURL == requests.utils.urlparse(link)[1]:     
            if link not in [_[0] for _ in URLs] and \
                link not in [_[0] for _ in visitedURLs]:
                URLs.append((link, depth))
        if depth < useDepth:
            if link not in [_[0] for _ in URLs] and \
                link not in [_[0] for _ in visitedURLs]:
                URLs.append((link, depth))
    
    print(len(URLs), len(visitedURLs))
    
#     if requests.utils.urlparse(url)[1]  == lastURL:
#         time.sleep(random.randint(5, 10))
    
    lastURL = requests.utils.urlparse(url)[1] 

https://blog.naver.com/noemisuh?Redirect=Log&logNo=221628595728
24 1
https://blog.naver.com/FILEPATH
404
Not Found
{'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}
https://blog.naver.com/FILEPATH
https://blog.naver.com/PostList.nhn?blogId=noemisuh&categoryNo=162&from=postList&parentCategoryNo=0
23 3
https://blog.naver.com/noemisuh/221641577501
23 4
https://blog.naver.com/noemisuh/FILEPATH
404
Not Found
{'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}
https://blog.naver.com/noemisuh/FILEPATH
https://blog.naver.com/noemisuh/221628595728
21 6
https://blog.naver.com/PostList.nhn?blogId=noemisuh&categoryNo=162&from=postList&parentCategoryNo=162
20 7
https://blog.na

163 68
http://blog.naver.com/PostList.nhn?from=postList&blogId=tip944&categoryNo=28&currentPage=58
164 69
http://blog.naver.com/tip944/220862469207
166 70
http://blog.naver.com/tip944/FILEPATH
404
Not Found
{'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}
http://blog.naver.com/tip944/FILEPATH
http://blog.naver.com/PostList.nhn?blogId=tip944&categoryNo=19&from=postList&parentCategoryNo=0
164 72
http://blog.naver.com/PostList.nhn?blogId=tip944&categoryNo=19&from=postList
163 73
http://blog.naver.com/tip944/220862406325
165 74
http://blog.naver.com/PostList.nhn?blogId=tip944&categoryNo=20&from=postList&parentCategoryNo=0
164 75
http://blog.naver.com/PostList.nhn?blogId=tip944&categoryNo=18&parentCategoryNo=18&from=postList
173 76
http://blog.naver.com/PostList.nhn?from=postList&blogId=tip944&categoryNo=18&parentCategoryNo=18&

229 147
https://blog.naver.com/PostView.nhn?blogId=tip944&logNo=220862469207&categoryNo=19&parentCategoryNo=18&viewDate=&currentPage=&postListTopCurrentPage=&isAfterWrite=true
229 148
https://blog.naver.com/PostList.nhn?blogId=tip944&categoryNo=19&from=postList&parentCategoryNo=18
238 149
https://blog.naver.com/PostList.nhn?from=postList&blogId=tip944&categoryNo=19&parentCategoryNo=18&currentPage=11
247 150
https://blog.naver.com/PostList.nhn?from=postList&blogId=tip944&categoryNo=19&parentCategoryNo=18&currentPage=21
256 151
https://blog.naver.com/PostList.nhn?from=postList&blogId=tip944&categoryNo=19&parentCategoryNo=18&currentPage=31
265 152
https://blog.naver.com/PostList.nhn?from=postList&blogId=tip944&categoryNo=19&parentCategoryNo=18&currentPage=41
274 153
https://blog.naver.com/PostList.nhn?from=postList&blogId=tip944&categoryNo=19&parentCategoryNo=18&currentPage=51
281 154
https://blog.naver.com/PostList.nhn?from=postList&blogId=tip944&categoryNo=19&parentCategoryNo=18&current

282 216
https://blog.naver.com/PostList.nhn?from=postList&blogId=tip944&categoryNo=18&parentCategoryNo=18&currentPage=2
281 217
https://blog.naver.com/PostList.nhn?blogId=tip944&categoryNo=20&from=postList
280 218
https://blog.naver.com/PostList.nhn?from=postList&blogId=tip944&categoryNo=19&parentCategoryNo=18&currentPage=58
279 219
https://blog.naver.com/PostList.nhn?from=postList&blogId=tip944&categoryNo=19&parentCategoryNo=18&currentPage=57
278 220
https://blog.naver.com/PostList.nhn?from=postList&blogId=tip944&categoryNo=19&parentCategoryNo=18&currentPage=56
277 221
https://blog.naver.com/PostList.nhn?from=postList&blogId=tip944&categoryNo=19&parentCategoryNo=18&currentPage=55
276 222
https://blog.naver.com/PostList.nhn?from=postList&blogId=tip944&categoryNo=19&parentCategoryNo=18&currentPage=54
275 223
https://blog.naver.com/PostList.nhn?from=postList&blogId=tip944&categoryNo=19&parentCategoryNo=18&currentPage=53
274 224
https://blog.naver.com/PostList.nhn?from=postList&blogId=tip

274 284
https://blog.naver.com/PostList.nhn?from=postList&blogId=tip944&categoryNo=20&parentCategoryNo=18&currentPage=53
273 285
https://blog.naver.com/PostList.nhn?from=postList&blogId=tip944&categoryNo=20&parentCategoryNo=18&currentPage=52
272 286
https://blog.naver.com/PostList.nhn?from=postList&blogId=tip944&categoryNo=20&parentCategoryNo=18&currentPage=50
271 287
https://blog.naver.com/PostList.nhn?from=postList&blogId=tip944&categoryNo=20&parentCategoryNo=18&currentPage=49
270 288
https://blog.naver.com/PostList.nhn?from=postList&blogId=tip944&categoryNo=20&parentCategoryNo=18&currentPage=48
269 289
https://blog.naver.com/PostList.nhn?from=postList&blogId=tip944&categoryNo=20&parentCategoryNo=18&currentPage=47
268 290
https://blog.naver.com/PostList.nhn?from=postList&blogId=tip944&categoryNo=20&parentCategoryNo=18&currentPage=46
267 291
https://blog.naver.com/PostList.nhn?from=postList&blogId=tip944&categoryNo=20&parentCategoryNo=18&currentPage=45
266 292
https://blog.naver.com/P

...

182 3329
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&logType=2&tagName=%EC%84%9D%EB%B6%84
181 3330
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&tagName=%EC%84%9D%EB%B6%84
180 3331
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&logType=0&tagName=%EC%A0%95%ED%99%94%EC%A1%B0%ED%83%B1%ED%81%AC%EB%A7%A4%EB%A6%BD
181 3332
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&logType=2&tagName=%EC%A0%95%ED%99%94%EC%A1%B0%ED%83%B1%ED%81%AC%EB%A7%A4%EB%A6%BD
180 3333
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&tagName=%EC%A0%95%ED%99%94%EC%A1%B0%ED%83%B1%ED%81%AC%EB%A7%A4%EB%A6%BD
179 3334
http://blog.naver.com/tip944/220758304588
178 3335
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&logType=2&tagName=%ED%8F%B4%EB%A6%AC%EC%8B%B1%ED%83%80%EC%9D%BC
177 3336
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&tagName=%ED%8F%B4%EB%A6%AC%EC%8B%B1%ED%83%80%EC%9D%BC
176 3337
http://blog.naver.com/tip944/220755994000
175 3338
ht

168 3400
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&logType=0&tagName=701%EB%B2%88%EB%B2%84%EC%8A%A4
169 3401
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&logType=2&tagName=701%EB%B2%88%EB%B2%84%EC%8A%A4
168 3402
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&tagName=701%EB%B2%88%EB%B2%84%EC%8A%A4
167 3403
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&logType=0&tagName=15%ED%86%A4%EC%A0%95%ED%99%94%EC%A1%B0%ED%83%B1%ED%81%AC
168 3404
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&logType=2&tagName=15%ED%86%A4%EC%A0%95%ED%99%94%EC%A1%B0%ED%83%B1%ED%81%AC
167 3405
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&tagName=15%ED%86%A4%EC%A0%95%ED%99%94%EC%A1%B0%ED%83%B1%ED%81%AC
166 3406
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&logType=0&tagName=108%EB%B0%B0
168 3407
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&logType=0&tagName=%EC%A0%88%EC%88%98%EB%A0%A8
169 3408
http://blog.naver.com/Post

176 3467
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&tagName=%EC%A7%80%EB%B6%95%EA%B3%B5%EC%82%AC
175 3468
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&logType=0&tagName=%EC%8A%A4%ED%8C%A8%EB%8B%88%EC%8B%9C%EA%B8%B0%EC%99%80
176 3469
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&logType=2&tagName=%EC%8A%A4%ED%8C%A8%EB%8B%88%EC%8B%9C%EA%B8%B0%EC%99%80
175 3470
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&tagName=%EC%8A%A4%ED%8C%A8%EB%8B%88%EC%8B%9C%EA%B8%B0%EC%99%80
174 3471
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&logType=0&tagName=%EC%99%B8%EB%B2%BD%EB%93%9C%EB%9D%BC%EC%9D%B4%EB%B9%84%ED%8A%B8
175 3472
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&logType=2&tagName=%EC%99%B8%EB%B2%BD%EB%93%9C%EB%9D%BC%EC%9D%B4%EB%B9%84%ED%8A%B8
174 3473
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&tagName=%EC%99%B8%EB%B2%BD%EB%93%9C%EB%9D%BC%EC%9D%B4%EB%B9%84%ED%8A%B8
173 3474
http://blog.naver.com/tip944/220

159 3535
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&logType=2&tagName=%EC%98%88%EC%95%BD%ED%99%95%EC%9D%B8%EC%82%AC%ED%95%AD
158 3536
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&tagName=%EC%98%88%EC%95%BD%ED%99%95%EC%9D%B8%EC%82%AC%ED%95%AD
157 3537
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&logType=0&tagName=%ED%99%98%EB%B6%88%EA%B7%9C%EC%A0%95
158 3538
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&logType=2&tagName=%ED%99%98%EB%B6%88%EA%B7%9C%EC%A0%95
157 3539
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&tagName=%ED%99%98%EB%B6%88%EA%B7%9C%EC%A0%95
156 3540
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&logType=0&tagName=%EC%A0%9C%EC%A3%BC%EB%8F%84%ED%95%A8%EB%8D%95
158 3541
http://blog.naver.com/tip944/220818073055
157 3542
http://blog.naver.com/PostListByTagName.nhn?blogId=tip944&logType=2&tagName=%EC%A0%9C%EC%A3%BC%EB%8F%84%ED%95%A8%EB%8D%95
156 3543
http://blog.naver.com/PostListByTagName.nhn?blogId=t

KeyboardInterrupt: 